<a href="https://colab.research.google.com/github/thibaudju/velyon/blob/main/python/V%C3%A9lyon_Pr%C3%A9diction_trafic_Prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Récupération des données sur Bigquery




In [1]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
client = bigquery.Client(project='velyon-batch-1187')

In [2]:
# Use LIMIT to make your tests if table is very large. Remove it to get the final table
sql_query = ('''SELECT *
                FROM velyon-batch-1187.dbt_prod.mart_compteurs_daily
                ''')

df = client.query(sql_query).to_dataframe()

In [4]:
import pandas as pd
df["day"] = pd.to_datetime(df.day).dt.tz_localize(None)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4533 entries, 0 to 4532
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   day                   4533 non-null   datetime64[ns]
 1   nb_passages           4533 non-null   Int64         
 2   avg_normalized_count  4533 non-null   float64       
dtypes: Int64(1), datetime64[ns](1), float64(1)
memory usage: 110.8 KB


Let's plot trafic over time to see if there is seasonality. Since our trafic is already normalized to account for variations in counter number or base volume of trafic for each location we won't have to do that.

1.   Élément de liste
2.   Élément de liste



In [6]:
import plotly.express as px
df = df.sort_values(by="day")
fig = px.line(df, "day", "avg_normalized_count", )
fig.show()

## Using PROPHET model

In [7]:
!pip install greykite -q
!pip install holidays==0.24
!pip install prophet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.5/22.5 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.9/499.9 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 75.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
from prophet import Prophet

We want to use the PROPHET model from Facebook to train our model and make a prediction for future turnover.

In [12]:
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.templates.autogen.forecast_config import (
    ComputationParam, EvaluationMetricParam, EvaluationPeriodParam,
    ForecastConfig, MetadataParam, ModelComponentsParam)
from greykite.framework.templates.forecaster import Forecaster
import datetime as dt
from datetime import datetime

config = ForecastConfig(
    metadata_param=MetadataParam(
        time_col="day",
        value_col="avg_normalized_count",
        freq="D",
        train_end_date=dt.datetime(2022, 12, 31)
    ),
    model_template=ModelTemplateEnum.PROPHET.name,
    forecast_horizon=365,
    coverage=0.95,

)

In [13]:
forecaster = Forecaster()
result = forecaster.run_forecast_config(df=df, config=config)
# model_prophet = result.model

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


DEBUG:cmdstanpy:input tempfile: /tmp/tmpnllcpgcw/hjaxry9m.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnllcpgcw/e3wu7eti.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=33270', 'data', 'file=/tmp/tmpnllcpgcw/hjaxry9m.json', 'init=/tmp/tmpnllcpgcw/e3wu7eti.json', 'output', 'file=/tmp/tmpnllcpgcw/prophet_modelqmu83djx/prophet_model-20230614081739.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
08:17:39 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
08:17:39 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnllcpgcw/f4rzss_m.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnllcpgcw/tcqq6ng0.json
DEBUG:cmdstanpy:

In [14]:
result.forecast.plot()

Let's try on a weekly basis

In [ ]:
# Aggregate daily traffic data to weekly data
df['day'] = pd.to_datetime(df['day'])  # Convert 'day' column to datetime if it's not already
df_weekly = df.resample('W-Mon', on='day').mean()  # Resample the data to weekly, starting on Mondays

# Reset the index of the new dataframe
df_weekly = df_weekly.reset_index()

# Rename the columns if needed
df_weekly = df_weekly.rename(columns={'day': 'ds', 'avg_normalized_count': 'y'})

# Prepare the data for Prophet model
prophet_df = pd.DataFrame()
prophet_df['ds'] = df_weekly['ds']
prophet_df['y'] = df_weekly['y']

In [ ]:
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.templates.autogen.forecast_config import (
    ComputationParam, EvaluationMetricParam, EvaluationPeriodParam,
    ForecastConfig, MetadataParam, ModelComponentsParam)
from greykite.framework.templates.forecaster import Forecaster
import datetime as dt
from datetime import datetime

config = ForecastConfig(
    metadata_param=MetadataParam(
        time_col="ds",
        value_col="y",
        freq="W-MON",
        train_end_date=datetime.datetime(2023, 1, 2)
    ),
    model_template=ModelTemplateEnum.PROPHET.name,
    forecast_horizon=52,
    coverage=0.95,

)

In [ ]:
forecaster = Forecaster()
result = forecaster.run_forecast_config(df=prophet_df, config=config)
# model_prophet = result.model

In [ ]:
result.forecast.plot()